In [1]:
# This script is meant to convert the NLDAS data into NetCDF format
# This will be used by the MetSim algorithm to create the radiation/vapor pressure

In [2]:
"""
Modified on Fri June 5 2020
@author: Jonathan Frame
"""

'\nModified on Fri June 5 2020\n@author: Jonathan Frame\n'

In [3]:
import datetime as dt
import pandas as pd
import numpy as np
import xarray as xr
import glob
from metsim import MetSim
import netCDF4 as nc
import re

In [48]:
main_dir = '/home/NearingLab/projects/jmframe/CAMELS_synthData/nldas_forcing/'
data_dir_txt = '/home/NearingLab/projects/jmframe/CAMELS_synthData/nldas_forcing_txt/'
data_dir_nc = '/home/NearingLab/projects/jmframe/CAMELS_synthData/nldas_forcing_nc/'
state_dir_nc = '/home/NearingLab/projects/jmframe/CAMELS_synthData/state_files/'

In [5]:
#Store all attributes. Really just need basin area, but the rest might come in handy.
att_path = "/home/NearingLab/data/camels_attributes_v2.0/camels_all.txt"
attributes = pd.read_csv(att_path, sep=";")
attributes.set_index('gauge_id', inplace=True)

In [6]:
# Get the prefixes for all the forcing data files.
prefix_dict = {'{:02}'.format(pf):[] for pf in range(1, 19)}
for sub_dir_name in glob.glob('/home/NearingLab/projects/jmframe/CAMELS_synthData/nldas_forcing_txt/*'):
    if sub_dir_name == 'readme_git.md':
        continue
    for file_path in glob.glob(sub_dir_name+'/*'):
        prefix = sub_dir_name.split('/')[7]
        temp_basin_id = re.split('_|/', file_path)[11]
        if prefix in prefix_dict.keys():
            prefix_dict[prefix].append(temp_basin_id)
attributes['prefix'] = [np.nan]*attributes.shape[0]
for k in prefix_dict.keys():
    for b in prefix_dict[k]:
        attributes.loc[int(b), 'prefix'] = k

In [90]:
path_data_txt_sample = data_dir_txt + '01/01013500_lump_nldas_forcing_leap.txt'
data_txt_sample = pd.read_csv(path_data_txt_sample, engine='python', header=3, sep='\t')
nldas_time = [dt.datetime(1980, 1, 1) + dt.timedelta(days=day) 
                  for day in range(data_txt_sample.shape[0])]
nldas_doy = [nldas_time[t].timetuple().tm_yday for t in range(len(nldas_time))]
state_time = [dt.datetime(1979, 1, 1) + dt.timedelta(days=day) 
                  for day in range(366)]

In [37]:
def GET_LAT_LON_FROM_ATTRIBUTES(attributes, basin_id_int):
    return [attributes.loc[basin_id_int,'gauge_lat'], attributes.loc[basin_id_int,'gauge_lon']]

In [54]:
def SUMMARIZE_STATE_DATA(basinID):
    basin_id_int = int(basinID)
    basin_id_str = "{:08}".format(basin_id_int)
    latCatchment , lonCatchment = GET_LAT_LON_FROM_ATTRIBUTES(attributes, basin_id_int)
    
    # initialize the NetCDF forcing data file.
    data_dir_txt = '/home/NearingLab/projects/jmframe/CAMELS_synthData/nldas_forcing_txt/'
    data_dir_nc = '/home/NearingLab/projects/jmframe/CAMELS_synthData/nldas_forcing_nc/'
    prefix = attributes.loc[basin_id_int, 'prefix']
    path_data_txt = data_dir_txt + prefix + '/' + basin_id_str + '_lump_nldas_forcing_leap.txt'
    data_txt = pd.read_csv(path_data_txt, engine='python', header=3, sep='\t')
    data_txt['date_time'] = nldas_time
    data_txt['doy'] = nldas_doy
    data_ave = data_txt.groupby('doy').mean()
    return data_ave

In [93]:
def MAKE_STATE_FILES(basinID, data_averages):
    basin_id_int = int(basinID)
    basin_id_str = "{:08}".format(basin_id_int)
    latCatchment , lonCatchment = GET_LAT_LON_FROM_ATTRIBUTES(attributes, basin_id_int)

    #BEGIN #######################################################################
    #BEGIN #  WRITE DATA TO THE NETCDF state DATA FILE   #######################
    #BEGIN #######################################################################

    stateDataName = state_dir_nc + basin_id_str + '_state.nc'
    state = nc.Dataset(stateDataName, 'w', format='NETCDF4_CLASSIC')
    state.title = basin_id_str +' _state'
    state.description = 'State file for basin '+ basin_id_str + ' in the CAMELS catchments'

    state.createDimension('lat', 1)
    state.createDimension('lon', 1)
    state.createDimension('time', size=366)

    ### createVareables in new data set
    state.createVariable('Prec', np.int32, ('time',))
    state.variables['Prec'].units = 'mm'
    state.variables['Prec'].long_name = 'Precipitation'
    #state.variables['Prec'].v_type = 'scalarv'
    state.variables['Prec'][:] = np.array(data_averages['PRCP(mm/day)'])

    state.createVariable('Tmax', np.float32, ('time',))
    state.variables['Tmax'].units = 'C' 
    state.variables['Tmax'].longname = "Daily maximum temperature"
    #state.variables['Tmax'].v_type = 'scalarv'
    state.variables['Tmax'][:] = np.array(data_averages['Tmax(C)'])
    
    state.createVariable('Tmin', np.float32, ('time',))
    state.variables['Tmin'].units = 'C'
    state.variables['Tmin'].longname = "Daily minimum temperature"
    #state.variables['Tmin'].v_type = 'scalarv'
    state.variables['Tmin'][:] = np.array(data_averages['Tmin(C)'])
    
    state.createVariable('latitude', np.float32, ('lat',))
    state.variables['latitude'].units = 'degrees_north'
    state.variables['latitude'].long_name = 'latitude'
    state.variables['latitude'].axis = 'Y'
    state.variables['latitude'][:] = latCatchment

    state.createVariable('longitude', np.float32, ('lon',))
    state.variables['longitude'].units = 'degrees-east'
    state.variables['longitude'].long_name = 'longitude'
    state.variables['longitude'].axis = 'X'
    state.variables['longitude'][:] = lonCatchment

    state.createVariable('time', np.float64, ('time',))
    state.variables['time'].units = 'hours since 2000-01-01 00:00:00'
    state.variables['time'].long_name = "local time at grid location"
    state.variables['time'].calendar = 'standard'
    state.variables['time'][:] = nc.date2num(state_time, 
                                          units='hours since 2000-01-01 00:00', calendar='standard')
    
    state.close()

In [94]:
make_state_files = True
if make_state_files:
    for b in attributes.index.values:
        data_averages = SUMMARIZE_STATE_DATA(b)
        MAKE_STATE_FILES(b, data_averages)